In [1]:
import pandas as pd
import spacy
import PyPDF2

In [2]:
ROOT_FOLDER = '../../'

In [3]:
DATA_FOLDER = '../../data/'

In [4]:
DOWNLOAD_FOLDER = '../../downloads/'

In [5]:
downloaded_files_df = pd.read_parquet(f'{DATA_FOLDER}downloaded_files_df.parquet')

In [6]:
downloaded_files_df

,title,file_name
0,LncMachine: a machine learning algorithm for l...,51807917-91f3-4b8f-8ad4-e1c5c923432e.pdf
1,DMFLDA: a deep learning framework for predicti...,92c4c16b-1cc5-49b0-8ff3-09182bfb02fc.pdf
2,Evaluation of deep learning in non-coding RNA ...,d320ce0f-7cd8-4afa-8ec7-baad93b09505.pdf


In [7]:
downloaded_files_df['pdf_file_path'] = DOWNLOAD_FOLDER + downloaded_files_df['file_name']

In [8]:
downloaded_files_df

,title,file_name,pdf_file_path
0,LncMachine: a machine learning algorithm for l...,51807917-91f3-4b8f-8ad4-e1c5c923432e.pdf,../../downloads/51807917-91f3-4b8f-8ad4-e1c5c9...
1,DMFLDA: a deep learning framework for predicti...,92c4c16b-1cc5-49b0-8ff3-09182bfb02fc.pdf,../../downloads/92c4c16b-1cc5-49b0-8ff3-09182b...
2,Evaluation of deep learning in non-coding RNA ...,d320ce0f-7cd8-4afa-8ec7-baad93b09505.pdf,../../downloads/d320ce0f-7cd8-4afa-8ec7-baad93...


In [9]:
paper_titles = downloaded_files_df['title'].values.tolist()
paper_pdf_file_paths = downloaded_files_df['pdf_file_path'].values.tolist()

In [10]:
import spacy
from spacy.matcher import PhraseMatcher

# Load the EntityRuler patterns from a file
nlp = spacy.load("en_core_web_sm")
ruler = nlp.add_pipe("entity_ruler", before="ner")
ruler.from_disk(f"{DATA_FOLDER}ml_entity_ruler_patterns")

In [11]:
# Test the pipeline
text = "Support Vector Machine, Support Vector Machines, SVM, and S.V.M. are popular machine learning methods."
doc = nlp(text)

# Print detected entities
for ent in doc.ents:
    if ent.label_ == "ML_METHOD":
        print(ent.text, ent.label_)
        break

Support Vector Machine ML_METHOD


In [12]:
ent.text

'Support Vector Machine'

In [13]:
def search_in_file(pdf_file_path):
    search_results = dict()
    # Extract text from PDF
    with open(pdf_file_path, "rb") as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        for page_num, page in enumerate(pdf_reader.pages):
            # Extract text from the page
            text = page.extract_text()
            if text:
                # Use spaCy to process the text
                doc = nlp(text)
                
                # Detect entities
                for ent in doc.ents:
                    if ent.label_ == "ML_METHOD":
                        if ent.text in search_results:
                            search_results[ent.text].append(page_num + 1)  # Store page number (1-indexed)
                        else:
                            search_results[ent.text] = [page_num + 1]  # Store page number (1-indexed)
    
    # Display search results
    for term, pages in search_results.items():
        if pages:
            print(f"'{term}' found on page(s): {set(pages)}")
        #else:
        #    print(f"'{term}' not found in the document.")

In [14]:
for paper_title, pdf_file_path in zip(paper_titles, paper_pdf_file_paths):
    print('-'*80)
    print(f'processing: {paper_title}')
    print('-'*80)
    search_in_file(pdf_file_path=pdf_file_path)
    print('-'*80)

--------------------------------------------------------------------------------
processing: LncMachine: a machine learning algorithm for long noncoding RNA annotation in plants
--------------------------------------------------------------------------------
'Random Forest' found on page(s): {1, 3, 4, 9}
'Support Vector Machine' found on page(s): {2, 4, 5}
'SVM' found on page(s): {2, 4, 5}
'Logistic Regression' found on page(s): {2, 6}
'decision tree' found on page(s): {2}
'Random Forests' found on page(s): {2}
'boosting' found on page(s): {2}
'AdaBoost' found on page(s): {3, 4}
'AA' found on page(s): {4}
'Neural Networks' found on page(s): {9, 7}
'random forests' found on page(s): {9}
'support vector machine' found on page(s): {10}
'GB' found on page(s): {10}
'GMM' found on page(s): {10}
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
processing: DMFLDA: a deep learning fr

TODO:

- Experiment with using a simpler empty pipeline instead of en_core_web_sm or disable unwanted pipelines, see *Disabling pipeline components* in chapter 3 of learn-spaCy project
- Use *components with extensions* from chapter 3 of learn-spaCy project to look up the model category information e.g. training type